In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from utils.Points import PlayerPoints, PlayerServes
from utils.Builder import Builder
from utils.Tools import GetMatches
import numpy as np
import os
import csv
import warnings

In [23]:
warnings.filterwarnings('ignore')

In [24]:
SymbolsDictionary = {"F" : "Forehand", 
                     "B" : "Backhand", 
                     "R" :"FH Slice", 
                     "S" : "BH Slice", 
                     "V" : "FH Volley", 
                     "Z" : "BH Volley"}

EndDict = {"*": ("Winner","Ace"),
           "@": "Unforced Error",
           "#" : "Forced Error"}

In [25]:
# Builder('Roger Federer',(2005,2020))

In [26]:
    # directory_name = player +'_data'

    # try:
    #     os.mkdir(directory_name)
    #     print(f"Directory '{directory_name}' created successfully.")
    # except FileExistsError:
    #     print(f"Directory '{directory_name}' already exists.")
    # except PermissionError:
    #     print(f"Permission denied: Unable to create '{directory_name}'.")
    # except Exception as e:
    #     print(f"An error occurred: {e}")

In [27]:
# matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
# Base = pd.read_csv('BaseData/atp_matches_2012.csv',encoding='unicode_escape',quoting=csv.QUOTE_NONE)

In [28]:
    # matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)

    # matches2012 = matches[matches['Date'].str[:4] == '2012']
    # matches2012 = matches2012

    # FedererGames = matches2012[(matches2012['Player 1'] == 'Roger Federer') | (matches2012['Player 2'] == 'Roger Federer')]
    # NadalGames = matches2012[(matches2012['Player 1'] == 'Rafael Nadal') | (matches2012['Player 2'] == 'Rafael Nadal')]
    # DjokovicGames = matches2012[(matches2012['Player 1'] == 'Novak Djokovic') | (matches2012['Player 2'] == 'Novak Djokovic')]

    # Points = pd.read_csv('ProjData/Points.csv')

    # FedererAux = FedererGames[['match_id','Player 1', 'Player 2','Surface']]
    # NadalAux = NadalGames[['match_id','Player 1', 'Player 2','Surface']]
    # DjokovicAux = DjokovicGames[['match_id','Player 1', 'Player 2','Surface']]

    # PointsFederer = pd.merge(Points,FedererAux, on='match_id')
    # PointsNadal = pd.merge(Points,NadalAux, on='match_id')
    # PointsDjokovic = pd.merge(Points,DjokovicAux, on='match_id')

    # Victor(PointsFederer,FedererGames)
    # Victor(PointsDjokovic,DjokovicGames)
    # Victor(PointsNadal,NadalGames)

    # FedererGames.to_csv('ProjData/Federer2012.csv',index=False)
    # NadalGames.to_csv('ProjData/Nadal2012.csv',index=False)
    # DjokovicGames.to_csv('ProjData/Djokovic2012.csv',index=False)

    # PointsFederer['Victor'] = PtWinner(PointsFederer,'Roger Federer', FedererGames)
    # PointsNadal['Victor'] = PtWinner(PointsNadal,'Rafael Nadal', NadalGames)
    # PointsDjokovic['Victor'] = PtWinner(PointsDjokovic,'Novak Djokovic', DjokovicGames)

    # PointsFederer['Server'] = Server(PointsFederer,'Roger Federer', FedererGames)
    # PointsNadal['Server'] = Server(PointsNadal,'Rafael Nadal', NadalGames)
    # PointsDjokovic['Server'] = Server(PointsDjokovic,'Novak Djokovic', DjokovicGames)

    # PointsFederer[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None
    # PointsDjokovic[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None
    # PointsNadal[['Ace','Winner','Unforced Error','Forced Error','Double Fault']] = None

    # PointsFederer.loc[(PointsFederer['2nd'].isna()), ['2nd']] = None
    # PointsNadal.loc[(PointsNadal['2nd'].isna()), ['2nd']] = None
    # PointsDjokovic.loc[(PointsDjokovic['2nd'].isna()), ['2nd']] = None

    # PtEnding(PointsFederer,EndDict)
    # PtEnding(PointsDjokovic,EndDict)
    # PtEnding(PointsNadal,EndDict)

    # PointsFederer.to_csv('ProjData/PointsFederer.csv',index=False)
    # PointsDjokovic.to_csv('ProjData/PointsDjokovic.csv',index=False)
    # PointsNadal.to_csv('ProjData/PointsNadal.csv',index=False)

    # PointsFederer.to_csv('ProjData/PointsFederer.csv',index=False)
    # PointsDjokovic.to_csv('ProjData/PointsDjokovic.csv',index=False)
    # PointsNadal.to_csv('ProjData/PointsNadal.csv',index=False)

In [29]:
Federer = 'Roger Federer'
Nadal = 'Rafael Nadal'
Djokovic = 'Novak Djokovic'

In [30]:
matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
points = pd.read_csv('ProjData/Points.csv',encoding='unicode_escape',quoting=csv.QUOTE_NONE)


In [31]:
points

,match_id,Pt,Set1,Set2,Set#,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20091129-M-Tour_Finals-F-Nikolay_Davydenko-Jua...,1,0,0,1,0.0,0.0,0-0,1.0,True,1,4f2d#,NaN,NaN,1
1,20091129-M-Tour_Finals-F-Nikolay_Davydenko-Jua...,2,0,0,1,0.0,0.0,15-0,1.0,True,1,4n,4b27f3s2f+1f2n@,NaN,1
2,20091129-M-Tour_Finals-F-Nikolay_Davydenko-Jua...,3,0,0,1,0.0,0.0,30-0,1.0,True,1,4f29b2b2s1f1f2b2@,NaN,NaN,1
3,20091129-M-Tour_Finals-F-Nikolay_Davydenko-Jua...,4,0,0,1,0.0,0.0,40-0,1.0,True,1,4b29f2b1d@,NaN,NaN,1
4,20091129-M-Tour_Finals-F-Nikolay_Davydenko-Jua...,5,0,0,1,1.0,0.0,0-0,2.0,True,2,4r2d#,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988795,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,203,1,1,3,5.0,2.0,15-0,31.0,True,1,6w,5b39b2f1f1f2b2b1f1f2f2b2d@,NaN,1
988796,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,204,1,1,3,5.0,2.0,30-0,31.0,True,1,4n,4f28f3b2f1f2f+1r2v3*,NaN,1
988797,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,205,1,1,3,5.0,2.0,40-0,31.0,True,1,6f28b2b3b;2f1f1f3b;3s1f3w@,NaN,NaN,2
988798,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,206,1,1,3,5.0,2.0,40-15,31.0,True,1,5b37b2b1f2f2b2d@,NaN,NaN,2


In [33]:
def DivisionGames(Points,ids):
    GamesPerMatches = {}
    for id in ids:
        Match = Points[Points['match_id'] == id]
        
        Games_Matrix = []
        size = int(max(Match['Gm#']))
        
        for i in range(1,size + 1):
            Game = Match[Match['Gm#'] == i]

            Rallys = []
            for _,row in Game.iterrows():
            
                if(row['2nd'] == 'False'):
                    Rallys.append(row['1st'])
                else:
                    Rallys.append(row['2nd'])

            Games_Matrix.append(Rallys)
        
        GamesPerMatches[id] = Games_Matrix
    
    return GamesPerMatches


In [34]:
FedererGames = pd.read_csv('Federer_data/PointsFedererStart.csv')

In [35]:
GameDict = DivisionGames(FedererGames,FedererGames['match_id'].unique())

In [37]:
GameDict

{'20091128-M-Tour_Finals-SF-Roger_Federer-Nikolay_Davydenko': [['5b18f2b3b2b2f3s1f1*',
   '6*',
   '4r38f3b1d#',
   '4*'],
  ['6s38b2f2f2b1*',
   '4s39f1d@',
   '4f3n#',
   '4b38b2f2b2d@',
   '5b27b2f2b3b2b3s3f3b1f1f3n@',
   '4b1n@',
   '4f27f1f1f1f1f3s2n#',
   '6f28f3b1f1f3d@'],
  ['5d', '4b1w#', '4r27r-1d@', '5f27f+2b2z3*', '4w', '4b27f3b3f1w@'],
  ['6b28f2f+3s2v1*',
   '4s38f1f2n#',
   '6s37b2b2f3b3f3s3b3d@',
   '5s38b1r2n#',
   '4f28f3b3b3s2f1f2d#',
   '5f#'],
  ['6b28f2f3*', '4b27f1f2f2d@', '6n', '6r29f1n@'],
  ['4f28b3s3b1f2b+3m3*',
   '4s28f+1f2f+3s3z3*',
   '6s38b3b2f1f1w@',
   '4b38b3b1f2b1*',
   '4f29f3s3b2b3b3b2b1f2f1f1f1f2b2f1*',
   '4s39f3b2f1f1f1f2f3s3b3b3b2n#'],
  ['6s27f-1*',
   '4b37b3f1f2d@',
   '4b28f3b3b2f1f1d#',
   '6r27f+1f1v3b3*',
   '6*',
   '6f38b3w@'],
  ['6f18f3b3b3b1n@', '4s#', '6*', '4b37b1w@', '6b2n#'],
  ['4*', '5b18f3d#', '4*', '5s28f+1f2n#', '4r2n#'],
  ['4f18f2f3*',
   '4s38f1f2f3b3b3s2f3b2f1f1f1f2n#',
   '6f27b2f1f2f+3b3i1f1*',
   'c4b;28f1w@',
   '6s

In [40]:
pd.options.display.max_columns = None

In [ ]:
def DivisionSets(Points,ids):
    SetPerMatches = {}
    for id in ids:
        Match = Points[Points['match_id'] == id]
        Sets_Matrix = []
        size = int(max(Match['Set#']))

        for i in range(1,size + 1):
            Set = Match[Match['Set#'] == i]

            Rallys = []
            for _,row in Set.iterrows():
            
                if(row['2nd'] == 'False'):
                    Rallys.append(row['1st'])
                else:
                    Rallys.append(row['2nd']) 
                    
            Sets_Matrix.append(Rallys)
        
        SetPerMatches[id] = Sets_Matrix
    
    return SetPerMatches
            

In [53]:
SetDict = DivisionSets(FedererGames,FedererGames['match_id'].unique())

In [54]:
SetDict

{'20091128-M-Tour_Finals-SF-Roger_Federer-Nikolay_Davydenko': [['5b18f2b3b2b2f3s1f1*',
   'False',
   'False',
   'False',
   'False',
   '4s39f1d@',
   'False',
   'False',
   '5b27b2f2b3b2b3s3f3b1f1f3n@',
   '4b1n@',
   'False',
   '6f28f3b1f1f3d@',
   '5d',
   '4b1w#',
   'False',
   '5f27f+2b2z3*',
   '4w',
   '4b27f3b3f1w@',
   'False',
   '4s38f1f2n#',
   '6s37b2b2f3b3f3s3b3d@',
   '5s38b1r2n#',
   '4f28f3b3b3s2f1f2d#',
   'False',
   '6b28f2f3*',
   '4b27f1f2f2d@',
   '6n',
   'False',
   'False',
   'False',
   'False',
   '4b38b3b1f2b1*',
   'False',
   'False',
   'False',
   '4b37b3f1f2d@',
   '4b28f3b3b2f1f1d#',
   'False',
   'False',
   '6f38b3w@',
   '6f18f3b3b3b1n@',
   'False',
   'False',
   '4b37b1w@',
   'False'],
  ['False',
   '5b18f3d#',
   'False',
   'False',
   'False',
   'False',
   '4s38f1f2f3b3b3s2f3b2f1f1f1f2n#',
   '6f27b2f1f2f+3b3i1f1*',
   'False',
   'False',
   '4s38f1f1w#',
   'False',
   'False',
   'False',
   'False',
   'False',
   'False',
   '

In [ ]:
# start = 0
# Sets_Matrix = []
# size = len(Sets)
# for i in range(1,size):
#     idx = Sets.iloc[i]['Pt']
#     Set = FedererGame.iloc[start:(idx-1)]

#     Rallys = []
#     for _,row in Set.iterrows():

#         if(row['2nd'] == False):
#             Rallys.append(row['1st'])
#         else:
#             Rallys.append(row['2nd']) 
    
#     if(i == size-1):
#         First = FedererGame.iloc[len(FedererGame) - 1]['1st']
#         Second = FedererGame.iloc[len(FedererGame) - 1]['2nd']
#         if(Second == False):
#             Rallys.append(First)
#         else:
#             Rallys.append(Second) 

#     start = idx - 1
#     Sets_Matrix.append(Rallys)


    

In [16]:
Rally = pd.read_csv('BaseData/charting-m-stats-Rally.csv')
ShotTypes = pd.read_csv('BaseData\charting-m-stats-ShotTypes.csv')
Serves = pd.read_csv('BaseData/charting-m-stats-ServeBasics.csv')
ShotDir = pd.read_csv('BaseData/charting-m-stats-ShotDirection.csv')

In [ ]:
rowsServe = ['Total','1-3','4-6','7-9','10']
rowsIndx = ['match_id','1-3','4-6','7-9','10-']

In [ ]:
# text = '6n+f1f2b1b2-@'
# chars_to_remove = '+-'
# translation_table = str.maketrans('', '', chars_to_remove)
# updated_text = text.translate(translation_table)
# print(updated_text)

In [ ]:
rowsST = ['match_id','Total', 'F', 'B', 'R', 'S', 'V','Z']
FedererGamesShotTypes = FedererGamesShotTypes[FedererGamesShotTypes['row'].isin(rowsST)].reset_index(drop=True)
FedererShotTypes = FedererGamesShotTypes[FedererGamesShotTypes['player'] == 'Roger Federer']
FedererShotStats = Shots(FedererShotTypes,rows=rowsST)
FedererShotStats = FedererShotStats.rename(columns=SymbolsDictionary)
FedererShotStats = FedererShotStats.fillna(0)
FedererGames['Winner'] = FedererGames['Winner'].apply(lambda x: True if x == "['Roger Federer']" else False)
FedererShotStats['Win'] = FedererGames['Winner']
data = FedererShotStats[[ 'Total', 'Forehand', 'Backhand', 'FH Slice', 'BH Slice',
        'FH Volley', 'BH Volley', 'Win']]

In [ ]:

# F = forehand groundstroke (excluding slices, chips, etc.)
# b = backhand groundstroke (excluding slices, chips, etc.)

# r = forehand slice (including defensive chips, but not drop shots)
# s = backhand slice (including defensive chips, but not drop shots)
# v = forehand volley
# z = backhand volley
# (see below for an optional additional code to indicate a stop volley/drop volley)

# o = standard overhead/smash
# p = "backhand" overhead/smash
# u = forehand drop shot
# y = backhand drop shot
# l = forehand lob
# m = backhand lob
# h = forehand half-volley
# i = backhand half-volley
# j = forehand swinging volley
# k = backhand swinging volley
# t = all trick shots, including behind-the-back, between-the-legs, and "tweeners."


In [ ]:
Serves

,match_id,player,row,pts,pts_won,aces,unret,forced_err,pts_won_lte_3_shots,wide,body,t
0,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Camilo Ugo Carabelli,Total,77,45,5,0,10,19,27,32,18
1,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Camilo Ugo Carabelli,1,48,32,5,0,9,17,20,10,18
2,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Camilo Ugo Carabelli,2,29,13,0,0,1,2,7,22,0
3,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Karue Sell,Total,59,36,2,0,12,16,26,15,18
4,20241017-M-Campinas_CH-R16-Camilo_Ugo_Carabell...,Karue Sell,1,36,27,2,0,9,13,17,7,12
...,...,...,...,...,...,...,...,...,...,...,...,...
34855,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Nicola Pietrangeli,1,56,26,1,0,7,12,33,8,15
34856,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Nicola Pietrangeli,2,27,15,0,0,0,1,5,20,2
34857,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Luis Ayala,Total,85,41,2,0,6,13,41,15,29
34858,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,Luis Ayala,1,57,24,2,0,6,10,32,3,22


In [ ]:
FedererServes = ServStats(Serves,'Roger Federer',FedererGames)

In [ ]:
FedererServes

,match_id,Total,1st,Won1,2nd,Won2
0,20121112-M-Tour_Finals-F-Roger_Federer-Novak_D...,78,47,37,31,20
1,20121111-M-Tour_Finals-SF-Roger_Federer-Andy_M...,114,76,56,38,25
2,20121106-M-Tour_Finals-RR-Roger_Federer-Janko_...,95,63,53,32,18
3,20121028-M-Basel-F-Roger_Federer-Juan_Martin_D...,93,61,41,32,10
4,20121012-M-Shanghai_Masters-SF-Roger_Federer-A...,112,68,50,44,27
5,20121011-M-Shanghai_Masters-QF-Marin_Cilic-Rog...,157,100,68,57,33
6,20121008-M-Shanghai_Masters-R16-Roger_Federer-...,117,80,60,37,21
7,20120905-M-US_Open-QF-Tomas_Berdych-Roger_Federer,65,46,37,19,15
8,20120830-M-US_Open-R64-Roger_Federer-Bjorn_Phau,105,71,56,34,18
9,20120818-M-Cincinnati_Masters-F-Novak_Djokovic...,104,68,45,36,15


In [ ]:
PlayerServes = ServStats(Serves,'Roger Federer',FedererGames)

In [ ]:
PlayerServes

,match_id,Total,1st,Won1,2nd,Won2
0,20121112-M-Tour_Finals-F-Roger_Federer-Novak_D...,78,47,37,31,20
1,20121111-M-Tour_Finals-SF-Roger_Federer-Andy_M...,114,76,56,38,25
2,20121106-M-Tour_Finals-RR-Roger_Federer-Janko_...,95,63,53,32,18
3,20121028-M-Basel-F-Roger_Federer-Juan_Martin_D...,93,61,41,32,10
4,20121012-M-Shanghai_Masters-SF-Roger_Federer-A...,112,68,50,44,27
5,20121011-M-Shanghai_Masters-QF-Marin_Cilic-Rog...,157,100,68,57,33
6,20121008-M-Shanghai_Masters-R16-Roger_Federer-...,117,80,60,37,21
7,20120905-M-US_Open-QF-Tomas_Berdych-Roger_Federer,65,46,37,19,15
8,20120830-M-US_Open-R64-Roger_Federer-Bjorn_Phau,105,71,56,34,18
9,20120818-M-Cincinnati_Masters-F-Novak_Djokovic...,104,68,45,36,15
